## Water cycle percentage amplification analysis

The workflow to execute this notebook for a specific model/experiment can be run like this:
```
make pct -f pe_analysis.mk MODEL=IPSL-CM6A-LR EXPERIMENT=hist-GHG
```

In [ ]:
import os
import sys

import iris
import matplotlib.pyplot as plt
import numpy as np

cwd = os.getcwd()
repo_dir = '/'
for directory in cwd.split('/')[1:]:
    repo_dir = os.path.join(repo_dir, directory)
    if directory == 'ocean-analysis':
        break

modules_dir = os.path.join(repo_dir, 'modules')
sys.path.append(modules_dir)

import general_io as gio

In [1]:
# Input parameters

In [ ]:
# Required parameters
assert 'model_name' in locals(), "Must provide a model name"
assert 'experiment_name' in locals(), "Must provide an experiment name"
assert os.path.isfile(tasga_exp_file), "Must provide a global average surface temperature timeseries for the experiment (papermill option -p tasga_exp_file [filepath])"
assert os.path.isfile(tasga_cntrl_file), "Must provide a control global average surface temperature timeseries (papermill option -p tasga_cntrl_file [filepath])"
assert os.path.isfile(pe_cntrl_file), "Must provide a control P-E region sum timeseries (papermill option -p pe_cntrl_file [filepath])"
assert os.path.isfile(pe_exp_file), "Must provide a P-E region sum anomaly cumulative sum timeseries for the experiment (papermill option -p pe_exp_file [filepath])"

In [ ]:
tasga_exp_cube = iris.load_cube(tasga_exp_file, 'air_temperature')
tasga_cntrl_cube = iris.load_cube(tasga_cntrl_file, 'air_temperature')
pe_cntrl_cube = iris.load_cube(pe_cntrl_file, 'precipitation minus evaporation flux')
pe_exp_cube = iris.load_cube(pe_exp_file, 'precipitation minus evaporation flux')

In [ ]:
tas_units = str(tasga_exp_cube.units)

In [ ]:
pe_units = str(pe_cntrl_cube.units)

In [ ]:
years = np.arange(1861, 2006)

In [ ]:
time_constraint = gio.get_time_constraint(['1861-01-01', '2005-12-31'])

In [ ]:
tasga_exp_cube = tasga_exp_cube.extract(time_constraint)
pe_exp_cube = pe_exp_cube.extract(time_constraint)

In [ ]:
assert tasga_exp_cube.shape[0] == 145
assert pe_exp_cube.shape[0] == 145

## Global average temperature

In [ ]:
fig = plt.figure(figsize=(12, 4))
plt.plot(tasga_exp_cube.data, label=experiment_name)
plt.plot(tasga_cntrl_cube.data, label='piControl')
plt.title(f'Global average temperature, {model_name}')
plt.xlabel('Model year')
plt.ylabel(f'Temperature ({tas_units})')
plt.legend()
plt.show()

In [ ]:
baseline_tasga = tasga_cntrl_cube.data.mean()

In [ ]:
plt.plot(years, tasga_exp_cube.data, label=experiment_name)
plt.axhline(baseline_tasga, linestyle=':', label='piControl mean')
plt.title(f'Global average temperature, {model_name}')
plt.xlabel('Year')
plt.ylabel(f'Temperature ({tas_units})')
plt.legend()
plt.show()

In [ ]:
tasga_exp_anomaly = tasga_exp_cube.data - baseline_tasga
plt.plot(years, tasga_exp_anomaly)
plt.title(f'Global average temperature anomaly: {model_name}, {experiment_name} experiment')
plt.xlabel('Year')
plt.ylabel(f'Temperature anomaly ({tas_units})')
plt.show()

## 7% global P-E timeseries

In [ ]:
pe_cntrl_clim_cube = pe_cntrl_cube.collapsed('time', iris.analysis.MEAN)
pe_cntrl_clim_cube.remove_coord('time')

In [ ]:
# pereg: SH_precip SH_evap tropical_precip NH_evap NH_precip globe
# basin: atlantic pacific indian arctic marginal_seas land ocean globe        
ave_global_transport = pe_cntrl_clim_cube.data[0, -1] + pe_cntrl_clim_cube.data[2, -1] + pe_cntrl_clim_cube.data[4, -1]

In [ ]:
print(ave_global_transport)

In [ ]:
pe_cntrl_clim_cube.data[1, -1] + pe_cntrl_clim_cube.data[3, -1]

In [ ]:
sevenpct_anomaly_data = tasga_exp_anomaly * 0.07 * ave_global_transport

In [ ]:
fig = plt.figure(figsize=(12, 4))
plt.plot(years, sevenpct_anomaly_data)
plt.title(f'Inferred global moisture transport anomaly (for 7%/C): {model_name}, {experiment_name} experiment')
plt.xlabel('Year')
plt.ylabel(f'Transport anomaly ({pe_units})')
plt.show()

## Experiment versus 7%

In [ ]:
sevenpct_anomaly_cumsum_data = np.cumsum(sevenpct_anomaly_data) - sevenpct_anomaly_data[0]

In [ ]:
exp_cumsum_data = pe_exp_cube.data[:, 0, -1] + pe_exp_cube.data[:, 2, -1] + pe_exp_cube.data[:, 4, -1]
exp_anomaly_cumsum_data = exp_cumsum_data - exp_cumsum_data[0]

In [ ]:
fig = plt.figure(figsize=(8, 4))
plt.plot(years, sevenpct_anomaly_cumsum_data, label='7%')
plt.plot(years, exp_anomaly_cumsum_data, label=experiment_name)
plt.title(f'Cumulative global moisture transport anomaly, {model_name}')
plt.xlabel('Year')
plt.ylabel(f'cumulative anomaly ({pe_units})')
plt.legend()
plt.show()

In [ ]:
fraction_of_7pct = exp_anomaly_cumsum_data[-1] / sevenpct_anomaly_cumsum_data[-1]
fraction_of_7pct

In [ ]:
fraction_of_7pct * 7